In [1]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from collections import defaultdict
import os
import subprocess
from tqdm import trange
from copy import deepcopy

from env import Scenario, AMoD, CascadedQLearning
from util import mat2str, dictsum, moving_average

plt.style.use('ggplot')
CPLEXPATH = "C:/Program Files/ibm/ILOG/CPLEX_Studio1210/opl/bin/x64_win64/"

## Scenario 1 <br>

Given a 2x4 grid:

| 0 | 2 | 4 | 6 |<br>
| 1 | 3 | 5 | 7 | <br>

<br>

We assume the demand is generated according to K=2 periodic patterns. That is:


- K = 1 --> people go from 0 to 7 and from 6 to 1;
- K = 2 --> people go from 7 to 0 and from 1 to 6;

The RL algorithm outputs the desired distribution of vehicles in each region, which is then fed as input for the following ILP: (Notice added constraint compared to Frazzoli paper) 

\begin{equation}
\begin{aligned}
\min_{\alpha_{ij}} \quad & \sum_{i,j} T_{ij} \alpha_{ij}\\
\textrm{s.t.} \quad & \sum_{j \neq i} \alpha_{ji} - \alpha_{ij} \geq v_i^d - v_i, \quad \forall i \in \mathcal{N}\\
& \sum_{j \neq i} \alpha_{ij} \leq v_i, \quad \forall i \in \mathcal{N}\\
  &\alpha_{ij}\geq0, \quad \forall i \in \mathcal{N}\\
\end{aligned}
\end{equation}

In [2]:
scenario = Scenario()

In [3]:
env = AMoD(scenario)

In [4]:
# initialize agent
agent = CascadedQLearning(env=env)
num_nodes = len(agent.nodes)

In [2]:
# internal parameters for agent
params = {"training_round_len" : 600, # number of steps for which to train the same the same node (here 10 episodes of length 60) 
          "epsilon" : 1, # epsilon greedy initial parameter
          "k" : 0, # counter for Q table indexing (top-down learning)
          "default_action" : agent.action_space.index((0.5, 0.5))} # default behavior is to distribute vehicles equally

# learning parameters
alpha = 0.05 #learning rate                 
discount_factor = 0.9 # discount factor in Bellman Equation
# epsilon schedule parameters       
max_epsilon = 1
min_epsilon = 0.01         
its = 15
decays = [0.005]*5 + [0.01]*its + [0.03]*its + [0.05]*its

train_episodes = 10*7*len(decays) # num_of_episodes_with_same_epsilon x num_of_q_tables x num_epsilons          
max_steps = 100 # maximum length of episode
epochs = trange(train_episodes) # build tqdm iterator for loop visualization

# book-keeping variables
training_rewards = []
training_revenue = []
training_served_demand = []
training_rebalancing_cost = []

idx = (params["k"]//params["training_round_len"])%num_nodes # Q table index (initially, top-most node)
eps_idx = (params["k"]//(scenario.tf*num_nodes*10))%len(decays) # epsilon index (low decay to high decay schedules)
node = agent.nodes[idx] # select top-most node (i.e. (0, 1))
decay = decays[eps_idx] # select initial decay rate
switching_iters = np.linspace(0, (num_nodes-1)*params["training_round_len"], num_nodes) 

for episode in epochs:
    try:
        obs = env.reset()
        state = agent.encode_state(agent.decode_state(obs[0], obs[1])[idx])
        episode_reward = 0
        episode_revenue = 0
        episode_served_demand = 0
        episode_rebalancing_cost = 0
        for step in range(max_steps):
            idx = (params["k"]//params["training_round_len"])%num_nodes # Q table index (initially, top-most node)
            eps_idx = (params["k"]//(scenario.tf*num_nodes*10))%len(decays) # epsilon index (low decay to high decay schedules)
            node = agent.nodes[idx] # select node (i.e. (0, 1))
            decay = decays[eps_idx] # select decay rate
            
            action, action_rl = agent.policy(obs, params, train=True, CPLEXPATH=CPLEXPATH, res_path="S1/")
    
            # Take step
            new_obs, reward, done, info = env.step(action, isMatching=False)
            new_state = agent.encode_state(agent.decode_state(new_obs[0], new_obs[1])[idx])

            # Update Q table - Bellman Equation
            action_q = action_rl[idx] # select action from Q table currently training
            agent.Q[node][state, action_q] = agent.Q[node][state, action_q] + alpha*(reward+discount_factor*np.max(agent.Q[node][new_state, :])-agent.Q[node][state, action_q])

            # track performance over episode
            episode_reward += reward
            episode_revenue += info['revenue']
            episode_served_demand += info['served_demand']
            episode_rebalancing_cost += info['rebalancing_cost']
            obs, state = deepcopy(new_obs), deepcopy(new_state)

            # update parameters
            params["epsilon"] = min_epsilon+(max_epsilon-min_epsilon)*np.exp(-decay*step)
            params["k"] += 1
            # end episode if conditions reached
            if done:
                break
            # checkpointing
#             if (k)%59==0:
#                 np.save("./saved_files/agents/checkpoint/q_v1", agent.Q)
#                 np.save("./saved_files/agents/checkpoint/reward_v1", training_rewards)
#                 np.save("./saved_files/agents/checkpoint/revenue_v1", training_revenue)
#                 np.save("./saved_files/agents/checkpoint/served_demand_v1", training_served_demand)
#                 np.save("./saved_files/agents/checkpoint/cost_v1", training_rebalancing_cost)
            
        epochs.set_description(f"Episode {episode+1} | Reward: {episode_reward:.2f} | Revenue: {episode_revenue:.2f} | ServedDemand: {episode_served_demand:.2f} \
        | Reb. Cost: {episode_rebalancing_cost:.2f} | Decay: {decay}, Idx: {idx}")
        #Adding the total reward and reduced epsilon values
        training_rewards.append(episode_reward)
        training_revenue.append(episode_revenue)
        training_served_demand.append(episode_served_demand)
        training_rebalancing_cost.append(episode_rebalancing_cost)
    except KeyboardInterrupt:
        break

### Test Episodes 

In [3]:
# Test Episodes
test_episodes = 100
epochs = trange(test_episodes) # build tqdm iterator for loop visualization
np.random.seed(10)

# book-keeping variables
test_rewards = []
test_revenue = []
test_served_demand = []
test_rebalancing_cost = []
test_operating_cost = []

for episode in epochs:
    try:
        obs = env.reset()
        episode_reward = 0
        episode_revenue = 0
        episode_served_demand = 0
        episode_rebalancing_cost = 0
        episode_operating_cost = 0
        for step in range(max_steps):
            action, action_rl = agent.policy(obs, params=dict(), train=False, CPLEXPATH=CPLEXPATH, res_path="S1/Test/")
    
            # Take step
            new_obs, reward, done, info = env.step(action, isMatching=False)

            # track performance over episode
            episode_reward += reward
            episode_revenue += info['revenue']
            episode_served_demand += info['served_demand']
            episode_rebalancing_cost += info['rebalancing_cost']
            episode_operating_cost += info['operating_cost']
            obs = deepcopy(new_obs)

            # end episode if conditions reached
            if done:
                break
            
        epochs.set_description(f"Episode {episode+1} | Reward: {episode_reward:.2f} | Revenue: {episode_revenue:.2f} | ServedDemand: {episode_served_demand:.2f} \
| Oper. Cost: {episode_operating_cost:.2f} | Decay: {decay}, Idx: {idx}")
        #Adding the total reward and reduced epsilon values
        test_rewards.append(episode_reward)
        test_revenue.append(episode_revenue)
        test_served_demand.append(episode_served_demand)
        test_rebalancing_cost.append(episode_rebalancing_cost)
    except KeyboardInterrupt:
        break

In [4]:
# Plot results
fig = plt.figure(figsize=(12,32))
fig.add_subplot(411)
plt.plot(training_rewards, label="Reward")
plt.title("Episode Rewards")
plt.xlabel("Episode")
plt.ylabel("J")
plt.legend()

fig.add_subplot(412)
plt.plot(training_revenue, label="Revenue")
plt.title("Episode Revenue")
plt.xlabel("Episode")
plt.ylabel("Revenue")
plt.legend()

fig.add_subplot(413)
plt.plot(training_served_demand, label="Served Demand")
plt.title("Episode Served Demand")
plt.xlabel("Episode")
plt.ylabel("Served Demand")
plt.legend()

fig.add_subplot(414)
plt.plot(training_rebalancing_cost, label="Reb. Cost")
plt.title("Episode Reb. Cost")
plt.xlabel("Episode")
plt.ylabel("Cost")
plt.legend()
plt.show()

print("Average Performance: \n")
print(f'Avg Reward: {np.mean(training_rewards):.2f}')
print(f'Total Revenue: {np.mean(training_revenue):.2f}')
print(f'Total Served Demand: {np.mean(training_served_demand):.2f}')
print(f'Total Rebalancing Cost: {np.mean(training_rebalancing_cost):.2f}')

In [5]:
# Plot results with moving average smoothing 
fig = plt.figure(figsize=(12,32))
fig.add_subplot(411)
plt.plot(moving_average(training_rewards, n=10), label="Avg. Reward")
plt.title("Avg. Rewards")
plt.xlabel("Episode")
plt.ylabel("J")
plt.legend()

fig.add_subplot(412)
plt.plot(moving_average(training_revenue, n=10), label="Avg. Revenue")
plt.title("Avg. Revenue")
plt.xlabel("Episode")
plt.ylabel("Revenue")
plt.legend()

fig.add_subplot(413)
plt.plot(moving_average(training_served_demand, n=10), label="Avg. Served Demand")
plt.title("Avg. Served Demand")
plt.xlabel("Episode")
plt.ylabel("Served Demand")
plt.legend()

fig.add_subplot(414)
plt.plot(moving_average(training_rebalancing_cost, n=10), label="Avg. Reb. Cost")
plt.title("Avg. Reb. Cost")
plt.xlabel("Episode")
plt.ylabel("Cost")
plt.legend()
plt.show()

In [6]:
# Plot results
print("Average Performance: \n")
print(f'Avg Reward: {np.mean(test_rewards):.2f}')
print(f'Total Revenue: {np.mean(test_revenue):.2f}')
print(f'Total Served Demand: {np.mean(test_served_demand):.2f}')
print(f'Total Rebalancing Cost: {np.mean(test_rebalancing_cost):.2f}')

## Scenario 2

Given a 2x4 grid:

| 0 | 2 | 4 | 6 |<br>
| 1 | 3 | 5 | 7 | <br>

<br>

We assume the demand is generated according to an unbalanced demand pattern (customers move from left to right). That is:


- K = {1,2} --> people go from 1 to 6 and from 0 to 7;

In [2]:
scenario = Scenario(demand_input = {(1,6):2, (0,7):2, 'default':0.1})

In [3]:
env = AMoD(scenario)

In [4]:
# initialize agent
agent = CascadedQLearning(env=env)
num_nodes = len(agent.nodes)

In [8]:
# internal parameters for agent
params = {"training_round_len" : 600, # number of steps for which to train the same the same node (here 10 episodes of length 60) 
          "epsilon" : 1, # epsilon greedy initial parameter
          "k" : 0, # counter for Q table indexing (top-down learning)
          "default_action" : agent.action_space.index((0.5, 0.5))} # default behavior is to distribute vehicles equally

# learning parameters
alpha = 0.05 #learning rate                 
discount_factor = 0.9 # discount factor in Bellman Equation
# epsilon schedule parameters       
max_epsilon = 1
min_epsilon = 0.01         
its = 15
decays = [0.005]*5 + [0.01]*its + [0.03]*its + [0.05]*its

train_episodes = 10*7*len(decays) # num_of_episodes_with_same_epsilon x num_of_q_tables x num_epsilons          
max_steps = 100 # maximum length of episode
epochs = trange(train_episodes) # build tqdm iterator for loop visualization

# book-keeping variables
training_rewards = []
training_revenue = []
training_served_demand = []
training_rebalancing_cost = []

idx = (params["k"]//params["training_round_len"])%num_nodes # Q table index (initially, top-most node)
eps_idx = (params["k"]//(scenario.tf*num_nodes*10))%len(decays) # epsilon index (low decay to high decay schedules)
node = agent.nodes[idx] # select top-most node (i.e. (0, 1))
decay = decays[eps_idx] # select initial decay rate
switching_iters = np.linspace(0, (num_nodes-1)*params["training_round_len"], num_nodes) 

for episode in epochs:
    try:
        obs = env.reset()
        state = agent.encode_state(agent.decode_state(obs[0], obs[1])[idx])
        episode_reward = 0
        episode_revenue = 0
        episode_served_demand = 0
        episode_rebalancing_cost = 0
        for step in range(max_steps):
            idx = (params["k"]//params["training_round_len"])%num_nodes # Q table index (initially, top-most node)
            eps_idx = (params["k"]//(scenario.tf*num_nodes*10))%len(decays) # epsilon index (low decay to high decay schedules)
            node = agent.nodes[idx] # select node (i.e. (0, 1))
            decay = decays[eps_idx] # select decay rate
            
            action, action_rl = agent.policy(obs, params, train=True, CPLEXPATH=CPLEXPATH, res_path="S2/")
    
            # Take step
            new_obs, reward, done, info = env.step(action, isMatching=False)
            new_state = agent.encode_state(agent.decode_state(new_obs[0], new_obs[1])[idx])

            # Update Q table - Bellman Equation
            action_q = action_rl[idx] # select action from Q table currently training
            agent.Q[node][state, action_q] = agent.Q[node][state, action_q] + alpha*(reward+discount_factor*np.max(agent.Q[node][new_state, :])-agent.Q[node][state, action_q])

            # track performance over episode
            episode_reward += reward
            episode_revenue += info['revenue']
            episode_served_demand += info['served_demand']
            episode_rebalancing_cost += info['rebalancing_cost']
            obs, state = deepcopy(new_obs), deepcopy(new_state)

            # update parameters
            params["epsilon"] = min_epsilon+(max_epsilon-min_epsilon)*np.exp(-decay*step)
            params["k"] += 1
            # end episode if conditions reached
            if done:
                break
            # checkpointing
#             if (k)%59==0:
#                 np.save("./saved_files/agents/checkpoint/q_v1", agent.Q)
#                 np.save("./saved_files/agents/checkpoint/reward_v1", training_rewards)
#                 np.save("./saved_files/agents/checkpoint/revenue_v1", training_revenue)
#                 np.save("./saved_files/agents/checkpoint/served_demand_v1", training_served_demand)
#                 np.save("./saved_files/agents/checkpoint/cost_v1", training_rebalancing_cost)
            
        epochs.set_description(f"Episode {episode+1} | Reward: {episode_reward:.2f} | Revenue: {episode_revenue:.2f} | ServedDemand: {episode_served_demand:.2f} \
        | Reb. Cost: {episode_rebalancing_cost:.2f} | Decay: {decay}, Idx: {idx}")
        #Adding the total reward and reduced epsilon values
        training_rewards.append(episode_reward)
        training_revenue.append(episode_revenue)
        training_served_demand.append(episode_served_demand)
        training_rebalancing_cost.append(episode_rebalancing_cost)
    except KeyboardInterrupt:
        break

### Test Episodes 

In [ ]:
# Test Episodes
test_episodes = 100
np.random.seed(10)

# book-keeping variables
test_rewards = []
test_revenue = []
test_served_demand = []
test_rebalancing_cost = []
test_operating_cost = []

for episode in epochs:
    try:
        obs = env.reset()
        episode_reward = 0
        episode_revenue = 0
        episode_served_demand = 0
        episode_rebalancing_cost = 0
        episode_operating_cost = 0
        for step in range(max_steps):
            action, action_rl = agent.policy(obs, params=dict(), train=False, CPLEXPATH=CPLEXPATH, res_path="S2/Test/")
    
            # Take step
            new_obs, reward, done, info = env.step(action, isMatching=False)

            # track performance over episode
            episode_reward += reward
            episode_revenue += info['revenue']
            episode_served_demand += info['served_demand']
            episode_rebalancing_cost += info['rebalancing_cost']
            episode_operating_cost += info['operating_cost']
            obs = deepcopy(new_obs)

            # end episode if conditions reached
            if done:
                break
            
        epochs.set_description(f"Episode {episode+1} | Reward: {episode_reward:.2f} | Revenue: {episode_revenue:.2f} | ServedDemand: {episode_served_demand:.2f} \
        | Reb. Cost: {episode_rebalancing_cost:.2f} | Decay: {decay}, Idx: {idx}")
        #Adding the total reward and reduced epsilon values
        test_rewards.append(episode_reward)
        test_revenue.append(episode_revenue)
        test_served_demand.append(episode_served_demand)
        test_rebalancing_cost.append(episode_rebalancing_cost)
    except KeyboardInterrupt:
        break

In [ ]:
# Plot results
fig = plt.figure(figsize=(12,32))
fig.add_subplot(411)
plt.plot(training_rewards, label="Reward")
plt.title("Episode Rewards")
plt.xlabel("Episode")
plt.ylabel("J")
plt.legend()

fig.add_subplot(412)
plt.plot(training_revenue, label="Revenue")
plt.title("Episode Revenue")
plt.xlabel("Episode")
plt.ylabel("Revenue")
plt.legend()

fig.add_subplot(413)
plt.plot(training_served_demand, label="Served Demand")
plt.title("Episode Served Demand")
plt.xlabel("Episode")
plt.ylabel("Served Demand")
plt.legend()

fig.add_subplot(414)
plt.plot(training_rebalancing_cost, label="Reb. Cost")
plt.title("Episode Reb. Cost")
plt.xlabel("Episode")
plt.ylabel("Cost")
plt.legend()
plt.show()

print("Average Performance: \n")
print(f'Avg Reward: {np.mean(training_rewards):.2f}')
print(f'Total Revenue: {np.mean(training_revenue):.2f}')
print(f'Total Served Demand: {np.mean(training_served_demand):.2f}')
print(f'Total Rebalancing Cost: {np.mean(training_rebalancing_cost):.2f}')

In [ ]:
# Plot results with moving average smoothing 
fig = plt.figure(figsize=(12,32))
fig.add_subplot(411)
plt.plot(moving_average(training_rewards, n=10), label="Avg. Reward")
plt.title("Avg. Rewards")
plt.xlabel("Episode")
plt.ylabel("J")
plt.legend()

fig.add_subplot(412)
plt.plot(moving_average(training_revenue, n=10), label="Avg. Revenue")
plt.title("Avg. Revenue")
plt.xlabel("Episode")
plt.ylabel("Revenue")
plt.legend()

fig.add_subplot(413)
plt.plot(moving_average(training_served_demand, n=10), label="Avg. Served Demand")
plt.title("Avg. Served Demand")
plt.xlabel("Episode")
plt.ylabel("Served Demand")
plt.legend()

fig.add_subplot(414)
plt.plot(moving_average(training_rebalancing_cost, n=10), label="Avg. Reb. Cost")
plt.title("Avg. Reb. Cost")
plt.xlabel("Episode")
plt.ylabel("Cost")
plt.legend()
plt.show()

In [ ]:
# Plot results
print("Average Performance: \n")
print(f'Avg Reward: {np.mean(test_rewards):.2f}')
print(f'Total Revenue: {np.mean(test_revenue):.2f}')
print(f'Total Served Demand: {np.mean(test_served_demand):.2f}')
print(f'Total Rebalancing Cost: {np.mean(test_rebalancing_cost):.2f}')